In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join("./EEG_To_Text.git"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
sys.path

['/home/linal20/NLP_final_project',
 '/home/linal20/.conda/envs/pytorch2/lib/python38.zip',
 '/home/linal20/.conda/envs/pytorch2/lib/python3.8',
 '/home/linal20/.conda/envs/pytorch2/lib/python3.8/lib-dynload',
 '',
 '/home/linal20/.local/lib/python3.8/site-packages',
 '/home/linal20/.conda/envs/pytorch2/lib/python3.8/site-packages',
 '/home/linal20/NLP_final_project/EEG_To_Text.git']

In [2]:


module_path = os.path.abspath(os.path.join("./EEG-to-Text-Project.git"))
if module_path not in sys.path:
    sys.path.append(module_path)
    
sys.path

['/home/linal20/NLP_final_project',
 '/home/linal20/.conda/envs/pytorch2/lib/python38.zip',
 '/home/linal20/.conda/envs/pytorch2/lib/python3.8',
 '/home/linal20/.conda/envs/pytorch2/lib/python3.8/lib-dynload',
 '',
 '/home/linal20/.local/lib/python3.8/site-packages',
 '/home/linal20/.conda/envs/pytorch2/lib/python3.8/site-packages',
 '/home/linal20/NLP_final_project/EEG_To_Text.git',
 '/home/linal20/NLP_final_project/EEG-to-Text-Project.git']

In [1]:
import nvidia_smi

nvidia_smi.nvmlInit()

deviceCount = nvidia_smi.nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(i)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    print("Device {}: {}, Memory : ({:.2f}% free): {}(total), {} (free), {} (used)".format(i, nvidia_smi.nvmlDeviceGetName(handle), 100*info.free/info.total, info.total, info.free, info.used))

nvidia_smi.nvmlShutdown()

Device 0: b'Tesla P100-PCIE-12GB', Memory : (99.27% free): 12884901888(total), 12790988800 (free), 93913088 (used)
Device 1: b'Tesla P100-PCIE-12GB', Memory : (99.27% free): 12884901888(total), 12790988800 (free), 93913088 (used)


In [4]:
torch.save(model.state_dict(), "NLP_final.pt")

In [14]:
# train_sentiment_textbased.py
# python3 train_sentiment_textbased.py \
#     --dataset_name SST \
#     --model_name pretrain_Bart \
#     --num_epoch 20 \
#     -lr 0.0001 \
#     -b 32 \
#     -s ./checkpoints/text_sentiment_classifier \
#     -cuda cuda:0
customed_cache_dir = "/home/linal20/transformers_cache"

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
import pickle
import json
import matplotlib.pyplot as plt
from glob import glob
import time
import copy
from tqdm import tqdm

from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, BartForSequenceClassification, BertTokenizer, BertConfig, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from data import ZuCo_dataset, SST_tenary_dataset
from model_sentiment import FineTunePretrainedTwoStep
from config import get_config
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    # preds: numpy array: N * 3 
    # labels: numpy array: N 
    pred_flat = np.argmax(preds, axis=1).flatten()  
    
    labels_flat = labels.flatten()
    
    # return np.sum(pred_flat == labels_flat) / len(labels_flat)

    return np.mean(pred_flat == labels_flat)

# def flat_accuracy_top_k(preds, labels, k):
#     topk_preds = [list(pred.argsort()[-k:][::-1]) for pred in preds]
#     right_count = sum(any(l in topk for l in label) for topk, label in zip(topk_preds, labels))
#     return right_count / len(labels)

# def flat_accuracy_top_k(preds, labels,k):
#     topk_preds = []
#     for pred in preds:
#         topk = pred.argsort()[-k:][::-1]
#         topk_preds.append(list(topk))
#     # print(topk_preds)
#     topk_preds = list(topk_preds)
#     right_count = 0
#     # print(len(labels))
#     for i in range(len(labels)):
#         l = labels[i][0]
#         if l in topk_preds[i]:
#             right_count+=1
#     return right_count/len(labels)

def train_model_SST(dataloaders, device, model, criterion, optimizer, scheduler, num_epochs=25, checkpoint_path_best = './checkpoints/text_sentiment_classifier/best/test.pt', checkpoint_path_last = './checkpoints/text_sentiment_classifier/last/test.pt'):
    # modified from: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
    since = time.time()
      
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000000000
    best_acc = 0.0
    

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'dev']:
            total_accuracy = 0.0
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Iterate over data.
            for input_ids,input_masks,sentiment_labels in tqdm(dataloaders[phase]):
                
                input_ids = input_ids.to(device)
                input_masks = input_masks.to(device)
                sentiment_labels = sentiment_labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                output = model(input_ids = input_ids, attention_mask = input_masks, return_dict = True, labels = sentiment_labels)
                logits = output.logits
                loss = output.loss
                # print(sentiment_labels)
                # backward + optimize only if in training phase
                if phase == 'train':
                    # with torch.autograd.detect_anomaly():
                    loss.backward()
                    optimizer.step()

                # calculate accuracy
                preds_cpu = logits.detach().cpu().numpy()
                label_cpu = sentiment_labels.cpu().numpy()

                total_accuracy += flat_accuracy(preds_cpu, label_cpu)

                # statistics
                running_loss += loss.item() * input_ids.size()[0] # batch loss
                # print('[DEBUG]loss:',loss.item())
                # print('#################################')
                

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = total_accuracy / len(dataloaders[phase])
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            print('{} Acc: {:.4f}'.format(phase, epoch_acc))

            # deep copy the model
            if phase == 'dev' and (epoch_acc > best_acc):
                best_loss = epoch_loss
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                '''save checkpoint'''
                torch.save(model.state_dict(), checkpoint_path_best)
                print(f'update best on dev checkpoint: {checkpoint_path_best}')
        print()

    print('Best val loss: {:4f}'.format(best_loss))
    print('Best val acc: {:4f}'.format(best_acc))
    torch.save(model.state_dict(), checkpoint_path_last)
    print(f'update last checkpoint: {checkpoint_path_last}')
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [6]:
args = get_config('train_sentiment_textbased')

usage: ipykernel_launcher.py [-h] -d DATASET_NAME -m MODEL_NAME -ne NUM_EPOCH
                             -lr LEARNING_RATE -b BATCH_SIZE -s SAVE_PATH
                             [-subj SUBJECTS] [-eeg EEG_TYPE]
                             [-band EEG_BANDS [EEG_BANDS ...]] [-cuda CUDA]
ipykernel_launcher.py: error: the following arguments are required: -d/--dataset_name, -m/--model_name, -ne/--num_epoch, -lr/--learning_rate, -b/--batch_size, -s/--save_path


SystemExit: 2

/home/linal20/.conda/envs/pytorch2/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
# if __name__ == '__main__':
#     args = get_config('train_sentiment_textbased')
customed_cache_dir = "/home/linal20/transformers_cache"

''' config param'''

# python3 train_sentiment_textbased.py \
#     --dataset_name SST \
#     --model_name pretrain_Bart \
#     --num_epoch 20 \
#     -lr 0.0001 \
#     -b 32 \
#     -s ./checkpoints/text_sentiment_classifier \
#     -cuda cuda:0


epoch_n = 20
# lr = 1e-3 # Bert, RoBerta
# lr = 1e-4 # Bart
# lr = args['learning_rate']
lr = 0.0001
dataset_name = 'SST'
dataset_setting = 'unique_sent'
batch_size = 32

# model_name = 'pretrain_Bert'
model_name = 'pretrain_RoBerta'
# model_name = 'pretrain_Bart'
print(f'model: {model_name}')

# save_path = args['save_path'] 
save_path = './checkpoints/text_sentiment_classifier'
 
dataset_name == 'SST'
save_name = f'Textbased_StanfordSentitmentTreeband_{model_name}_b{batch_size}_{epoch_n}_{lr}'

output_checkpoint_name_best = save_path + f'/best/{save_name}.pt' 
output_checkpoint_name_last = save_path + f'/last/{save_name}.pt' 


''' set random seeds '''
set_seed = 312
np.random.seed(set_seed)
torch.manual_seed(set_seed)
torch.cuda.manual_seed_all(set_seed)

''' set up device '''
# use cuda
if torch.cuda.is_available():  
     dev = "cuda:1"
else:  
    dev = "cpu"
device = torch.device(dev)
print(f'device: {dev}')


'''tokenizer'''
if model_name == 'pretrain_Bert':
    print('tokenizer: bert-base-cased')
    tokenizer = BertTokenizer.from_pretrained('bert-base-cased',cache_dir = customed_cache_dir)
elif model_name == 'pretrain_RoBerta':
    print('tokenizer: roberta-base')
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base',cache_dir = customed_cache_dir)
elif model_name == 'pretrain_Bart':
    print('tokenizer: bart-large')
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large',cache_dir = customed_cache_dir)

dataset_name == 'SST'

SST_SENTIMENT_LABELS = json.load(open('./dataset/stanfordsentiment/ternary_dataset.json'))
SST_dataset = SST_tenary_dataset(SST_SENTIMENT_LABELS, tokenizer)  

# train_size = int(0.9 * len(SST_dataset))
# val_size = len(SST_dataset) - train_size

# train_set, dev_set = random_split(SST_dataset, [train_size, val_size])
from torch.utils.data import random_split

train_size = int(0.9 * len(SST_dataset))
train_set, dev_set = random_split(SST_dataset, [train_size, len(SST_dataset) - train_size])


# print('{:>5,} training sample'.format(len(train_set)))
# print('{:>5,} validation samples'.format(len(dev_set)))


dataset_sizes = {'train': len(train_set), 'dev': len(dev_set)}
print('train size: ', len(train_set))
print('dev size: ', len(dev_set))

lr = 0.0001 
# train dataloader
train_dataloader = DataLoader(train_set, batch_size = batch_size, shuffle=True, num_workers=4)
# dev dataloader
val_dataloader = DataLoader(dev_set, batch_size = 1, shuffle=False, num_workers=4)
# dataloaders
dataloaders = {'train':train_dataloader, 'dev':val_dataloader}


if model_name == 'pretrain_Bert':
    model = BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=3,cache_dir = customed_cache_dir)
elif model_name == 'pretrain_RoBerta':
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3,cache_dir = customed_cache_dir)
elif model_name == 'pretrain_Bart':
    model = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels = 3,cache_dir = customed_cache_dir)

model.to(device)

# """save config"""
# with open(f'./config/text_sentiment_classifier/{save_name}.json', 'w') as out_config:
#         json.dump(args, out_config, indent = 4)

optimizer_step1 = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

exp_lr_scheduler_step1 = lr_scheduler.StepLR(optimizer_step1, step_size=10, gamma=0.1)

criterion = nn.CrossEntropyLoss()

# return best loss model 
print(f'=== start training using dataset {dataset_name} ===')
model = train_model_SST(dataloaders, device, model, criterion, optimizer_step1, exp_lr_scheduler_step1, num_epochs=epoch_n, checkpoint_path_best = output_checkpoint_name_best, checkpoint_path_last = output_checkpoint_name_last)    

model: pretrain_RoBerta
device: cuda:1
tokenizer: roberta-base


Original distribution:
	Very positive: 1701
	Neutral: 2101
	Very negative: 1383
balance class to 1383 each...
train size:  3734
dev size:  415


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== start training using dataset SST ===
Epoch 0/19
----------


100%|██████████| 117/117 [00:20<00:00,  5.69it/s]


...

dev Loss: 0.5549
dev Acc: 0.7759
update best on dev checkpoint: ./checkpoints/text_sentiment_classifier/best/Textbased_StanfordSentitmentTreeband_pretrain_RoBerta_b32_20_0.0001.pt

Epoch 19/19
----------


100%|██████████| 117/117 [00:20<00:00,  5.70it/s]


train Loss: 0.4202
train Acc: 0.8295


100%|██████████| 415/415 [00:06<00:00, 66.87it/s]


dev Loss: 0.5515
dev Acc: 0.7639

Best val loss: 0.554950
Best val acc: 0.775904
update last checkpoint: ./checkpoints/text_sentiment_classifier/last/Textbased_StanfordSentitmentTreeband_pretrain_RoBerta_b32_20_0.0001.pt


In [9]:
import json
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, random_split
import pickle
import json
import matplotlib.pyplot as plt
from glob import glob
import time
import copy
from tqdm import tqdm

from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, BartForSequenceClassification, BertTokenizer, BertConfig, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from data import ZuCo_dataset, SST_tenary_dataset
from model_sentiment import FineTunePretrainedTwoStep
from config import get_config
from data import ZuCo_dataset, SST_tenary_dataset
customed_cache_dir = "/home/linal20/transformers_cache"

tokenizer = BertTokenizer.from_pretrained('bert-base-cased',cache_dir = customed_cache_dir)

SST_SENTIMENT_LABELS = json.load(open('./dataset/stanfordsentiment/ternary_dataset.json'))
SST_dataset = SST_tenary_dataset(SST_SENTIMENT_LABELS, tokenizer)  

Original distribution:
	Very positive: 1701
	Neutral: 2101
	Very negative: 1383
balance class to 1383 each...


In [10]:
SST_dataset 

In [7]:
SST_SENTIMENT_LABELS

{"The gorgeously elaborate continuation of `` The Lord of the Rings '' trilogy is so huge that a column of words can not adequately describe co-writer\\/director Peter Jackson 's expanded vision of J.R.R. Tolkien 's Middle-earth .": 2,
 'Effective but too-tepid biopic': 1,
 "Emerges as something rare , an issue movie that 's so honest and keenly observed that it does n't feel like one .": 2,
 'The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .': 1,
 'Offers that rare combination of entertainment and education .': 2,
 'This is a film well worth seeing , talking and singing heads and all .': 2,
 'One of the greatest family-oriented , fantasy-adventure movies ever .': 2,
 'Ultimately , it ponders the reasons we need stories so much .': 1,
 'Illuminating if overly talky documentary .': 1,
 'A masterpiece four years in the making .': 2,
 'Offers a breath of the fresh air of true sophistication .': 2,
 'A

In [5]:
# eval_sentiment.py
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pack_padded_sequence 
import pickle
import json
import matplotlib.pyplot as plt
from glob import glob
import time
import copy
from tqdm import tqdm

from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig, BartForSequenceClassification, BertTokenizer, BertConfig, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from data import ZuCo_dataset
from model_sentiment import BaselineMLPSentence, BaselineLSTM, FineTunePretrainedTwoStep, ZeroShotSentimentDiscovery, JointBrainTranslatorSentimentClassifier
from model_decoding import BrainTranslator, BrainTranslatorNaive
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from config import get_config

customed_cache_dir = "/home/linal20/transformers_cache"

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    # preds: numpy array: N * 3 
    # labels: numpy array: N 
    pred_flat = np.argmax(preds, axis=1).flatten()  
    
    labels_flat = labels.flatten()
    
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_accuracy_top_k(preds, labels,k):
    topk_preds = []
    for pred in preds:
        topk = pred.argsort()[-k:][::-1]
        topk_preds.append(list(topk))
    # print(topk_preds)
    topk_preds = list(topk_preds)
    right_count = 0
    # print(len(labels))
    for i in range(len(labels)):
        l = labels[i][0]
        if l in topk_preds[i]:
            right_count+=1
    return right_count/len(labels)

def eval_model(dataloaders, device, model, criterion, optimizer, scheduler, num_epochs=25, tokenizer = BartTokenizer.from_pretrained('facebook/bart-large',cache_dir= customed_cache_dir)):

    def logits2PredString(logits, tokenizer):
        probs = logits[0].softmax(dim = 1)
        # print('probs size:', probs.size())
        values, predictions = probs.topk(1)
        # print('predictions before squeeze:',predictions.size())
        predictions = torch.squeeze(predictions)
        predict_string = tokenizer.decode(predictions)
        return predict_string
    
# modified from: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
    since = time.time()
      
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100000000000
    best_acc = 0.0
    
    total_pred_labels = np.array([])
    total_true_labels = np.array([])
    
    for epoch in range(1):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
    
        # Each epoch has a training and validation phase
        for phase in ['test']:
            total_accuracy = 0.0
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
    
            running_loss = 0.0
    
            # Iterate over data.
            for input_word_eeg_features, seq_lens, input_masks, input_mask_invert, target_ids, target_mask, sentiment_labels, sent_level_EEG in dataloaders[phase]:
                
                input_word_eeg_features = input_word_eeg_features.to(device).float()
                input_masks = input_masks.to(device)
                input_mask_invert = input_mask_invert.to(device)
    
                sent_level_EEG = sent_level_EEG.to(device)
                sentiment_labels = sentiment_labels.to(device)
    
                target_ids = target_ids.to(device)
                target_mask = target_mask.to(device)
    
                ## forward ###################
                if isinstance(model, BaselineMLPSentence):
                    logits = model(sent_level_EEG) # before softmax
                    # calculate loss
                    loss = criterion(logits, sentiment_labels)
    
                elif isinstance(model, BaselineLSTM):
                    x_packed = pack_padded_sequence(input_word_eeg_features, seq_lens, batch_first=True, enforce_sorted=False)
                    logits = model(x_packed)
                    # calculate loss
                    loss = criterion(logits, sentiment_labels)
    
                elif isinstance(model, BertForSequenceClassification) or isinstance(model, RobertaForSequenceClassification) or isinstance(model, BartForSequenceClassification):
                    output = model(input_ids = target_ids, attention_mask = target_mask, return_dict = True, labels = sentiment_labels)
                    logits = output.logits
                    loss = output.loss
                
                elif isinstance(model, FineTunePretrainedTwoStep):
                    output = model(input_word_eeg_features, input_masks, input_mask_invert, sentiment_labels)
                    logits = output.logits
                    loss = output.loss
    
                elif isinstance(model, ZeroShotSentimentDiscovery):    
                    print()
                    print('target string:',tokenizer.decode(target_ids[0]).replace('<pad>','').split('</s>')[0]) 
    
                    """replace padding ids in target_ids with -100"""
                    target_ids[target_ids == tokenizer.pad_token_id] = -100 
    
                    output = model(input_word_eeg_features, input_masks, input_mask_invert, target_ids, sentiment_labels)
                    logits = output.logits
                    loss = output.loss
                
                elif isinstance(model, JointBrainTranslatorSentimentClassifier):
    
                    print()
                    print('target string:',tokenizer.decode(target_ids[0]).replace('<pad>','').split('</s>')[0]) 
    
                    """replace padding ids in target_ids with -100"""
                    target_ids[target_ids == tokenizer.pad_token_id] = -100 
    
                    LM_output, classification_output = model(input_word_eeg_features, input_masks, input_mask_invert, target_ids, sentiment_labels)
                    LM_logits = LM_output.logits
                    print('pred string:', logits2PredString(LM_logits, tokenizer).split('</s></s>')[0].replace('<s>',''))
                    classification_loss = classification_output['loss']
                    logits = classification_output['logits']
                    loss = classification_loss 
                ###############################
    
                # backward + optimize only if in training phase
                if phase == 'train':
                    # with torch.autograd.detect_anomaly():
                    loss.backward()
                    optimizer.step()
    
                # calculate accuracy
                preds_cpu = logits.detach().cpu().numpy()
                label_cpu = sentiment_labels.cpu().numpy()
    
                total_accuracy += flat_accuracy(preds_cpu, label_cpu)
                
                # add to total pred and label array, for cal F1, precision, recall
                pred_flat = np.argmax(preds_cpu, axis=1).flatten()
                labels_flat = label_cpu.flatten()
    
                total_pred_labels = np.concatenate((total_pred_labels,pred_flat))
                total_true_labels = np.concatenate((total_true_labels,labels_flat))
                
    
                # statistics
                running_loss += loss.item() * sent_level_EEG.size()[0] # batch loss
                # print('[DEBUG]loss:',loss.item())
                # print('#################################')
                
    
            if phase == 'train':
                scheduler.step()
    
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = total_accuracy / len(dataloaders[phase])
            print('{} Loss: {:.4f}'.format(phase, epoch_loss))
            print('{} Acc: {:.4f}'.format(phase, epoch_acc))
    
            # deep copy the model
            if phase == 'test' and epoch_loss < best_loss:
                best_loss = epoch_loss
                best_acc = epoch_acc
        print()
    
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best test loss: {:4f}'.format(best_loss))
    print('Best test acc: {:4f}'.format(best_acc))
    print()
    print('test sample num:', len(total_pred_labels))
    print('total preds:',total_pred_labels)
    print('total truth:',total_true_labels)
    print('sklearn macro: precision, recall, F1:')
    print(precision_recall_fscore_support(total_true_labels, total_pred_labels, average='macro'))
    print()
    print('sklearn micro: precision, recall, F1:')
    print(precision_recall_fscore_support(total_true_labels, total_pred_labels, average='micro'))
    print()
    print('sklearn accuracy:')
    print(accuracy_score(total_true_labels,total_pred_labels))
    print()
    
    
    
if __name__ == '__main__':
# args = get_config('eval_sentiment')

 
    ''' config param'''
num_epochs = 1

dataset_setting = 'unique_sent'

'''model name'''
# model_name = 'BaselineMLP'
# model_name = 'BaselineLSTM'
# model_name = 'NaiveFinetuneBert'
# model_name = 'FinetunedBertOnText'
# model_name = 'FinetunedRoBertaOnText'
# model_name = 'FinetunedBartOnText'
model_name = 'ZeroShotSentimentDiscovery'
# model_name = args['model_name']

print(f'[INFO] eval {model_name}')
if model_name == 'ZeroShotSentimentDiscovery':
    '''load decoder and classifier config'''
    # config_decoder = json.load(open(args['decoder_config_path']))
    # config_classifier = json.load(open(args['classifier_config_path']))
    # config_decoder = json.load(open('./config/decoding/task1_finetune_BrainTranslator_skipstep1_b32_20_30_5e-05_5e-07_unique_sent.json'))
    # config_classifier = json.load(open('./config/text_sentiment_classifier/Textbased_StanfordSentitmentTreeband_pretrain_Bart_b32_20_0.0001.json'))
    '''choose generator'''
    decoder_name = 'BrainTranslator'
    # decoder_name = 'BrainTranslatorNaive'
    # decoder_name = config_decoder['model_name']
    # decoder_checkpoint = args['decoder_checkpoint_path']
    # decoder_checkpoint= './checkpoints/decoding/best/task1_finetune_BrainTranslator_skipstep1_b32_20_30_5e-05_5e-07_unique_sent.pt'
    # decoder_checkpoint= './checkpoints/decoding/best/final.pt'
    decoder_checkpoint='./save_data/checkpoints/task1_all_steps/best/final.pt'
    # decoder_checkpoint='./save_data/checkpoints/task1_all_steps/best/final.pt'
    
    print(f'[INFO] using decoder: {decoder_name}')
# /home/linal20/NLP_final_project/checkpoints/decoding/best
    '''choose classifier'''
    # pretrain_Bert, pretrain_RoBerta, pretrain_Bart
    # classifier_name = config_classifier['model_name']
    # classifier_name = 'pretrain_Bert'
    classifier_name = 'pretrain_RoBerta'
    # classifier_name = 'pretrain_Bart'
    # classifier_checkpoint = args['classifier_checkpoint_path']
    classifier_checkpoint = './checkpoints/text_sentiment_classifier/best/Textbased_StanfordSentitmentTreeband_pretrain_RoBerta_b32_20_0.0001.pt'
    # classifier_checkpoint = './checkpoints/text_sentiment_classifier/best/Textbased_StanfordSentitmentTreeband_pretrain_Bert_b32_20_0.0001.pt'
      # classifier_checkpoint = './checkpoints/text_sentiment_classifier/best/Textbased_StanfordSentitmentTreeband_pretrain_Bart_b32_20_0.0001.pt'
    print(f'[INFO] using classifier: {classifier_name}')
else:
    checkpoint_path = args['checkpoint_path']
    # checkpoint_path = './checkpoints/text_sentiment_classifier/best/Textbased_StanfordSentitmentTreeband_pretrain_Bart_b32_20_0.0001.pt'
    print('[INFO] loading baseline:', checkpoint_path)

batch_size = 1


subject_choice = 'ALL'
# subject_choice = args['subjects']
print(f'![Debug]using {subject_choice}')
eeg_type_choice = 'GD'
# eeg_type_choice = args['eeg_type']
print(f'[INFO]eeg type {eeg_type_choice}')
# bands_choice = ['_t1'] 
bands_choice = ['_t1','_t2','_a1','_a2','_b1','_b2','_g1','_g2'] 
# bands_choice = args['eeg_bands']
print(f'[INFO]using bands {bands_choice}')



''' set random seeds '''
seed_val = 312
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


''' set up device '''
# use cuda
if torch.cuda.is_available():  
    # dev = args['cuda']
     dev = 'cuda:1'
else:  
    dev = "cpu"
# CUDA_VISIBLE_DEVICES=0,1,2,3  
device = torch.device(dev)
print(f'[INFO]using device {dev}')


''' load pickle'''
whole_dataset_dict = []
dataset_path_task1 = './dataset/ZuCo/task1-SR/pickle/Task1_SR_processed.pickle' 
with open(dataset_path_task1, 'rb') as handle:
    whole_dataset_dict.append(pickle.load(handle))

# '''set up tokenizer'''
# if model_name in ['BaselineMLP','BaselineLSTM', 'NaiveFinetuneBert', 'FinetunedBertOnText']:
#     print('[INFO]using Bert tokenizer')
#     tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# elif model_name == 'FinetunedBartOnText':
#     print('[INFO]using Bart tokenizer')
#     tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# elif model_name == 'FinetunedRoBertaOnText':
#     print('[INFO]using RoBerta tokenizer')
#     tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')
if model_name == 'ZeroShotSentimentDiscovery':
   decoder_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large',cache_dir = customed_cache_dir ) # Bart
   tokenizer = decoder_tokenizer
   if classifier_name == 'pretrain_Bert':
      sentiment_tokenizer = BertTokenizer.from_pretrained('bert-base-cased',cache_dir=customed_cache_dir) # Bert
   elif classifier_name == 'pretrain_Bart':
        sentiment_tokenizer = decoder_tokenizer
   elif classifier_name == 'pretrain_RoBerta':
        sentiment_tokenizer = RobertaTokenizer.from_pretrained('roberta-base',cache_dir=customed_cache_dir)

# ''' set up model '''
# if model_name == 'BaselineMLP':
#     print('[INFO]Model: BaselineMLP')
#     model = BaselineMLPSentence(input_dim = 840, hidden_dim = 128, output_dim = 3)
# elif model_name == 'BaselineLSTM':
#     print('[INFO]Model: BaselineLSTM')
#     # model = BaselineLSTM(input_dim = 840, hidden_dim = 256, output_dim = 3, num_layers = 1)
#     model = BaselineLSTM(input_dim = 840, hidden_dim = 256, output_dim = 3, num_layers = 4)
# elif model_name == 'FinetunedBertOnText':
#     print('[INFO]Model: FinetunedBertOnText')
#     model = BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=3)
# elif model_name == 'FinetunedRoBertaOnText':
#     print('[INFO]Model: FinetunedRoBertaOnText')
#     model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
# elif model_name == 'FinetunedBartOnText':
#     print('[INFO]Model: FinetunedBartOnText')
#     model = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=3)
if model_name == 'ZeroShotSentimentDiscovery':
   print(f'[INFO]Model: ZeroShotSentimentDiscovery, using classifer:{classifier_name}, using generator: {decoder_name}')
   pretrained = BartForConditionalGeneration.from_pretrained('facebook/bart-large',cache_dir = customed_cache_dir)
   if decoder_name == 'BrainTranslator':
      decoder = BrainTranslator(pretrained, in_feature = 105*len(bands_choice), decoder_embedding_size = 1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward = 2048)
    # elif decoder_name == 'BrainTranslatorNaive':
    #     decoder = BrainTranslatorNaive(pretrained, in_feature = 105*len(bands_choice), decoder_embedding_size = 1024, additional_encoder_nhead=8, additional_encoder_dim_feedforward = 2048)
    
   decoder.load_state_dict(torch.load(decoder_checkpoint), strict=False)
    
   if classifier_name == 'pretrain_Bert':
      classifier = BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels=3,cache_dir=customed_cache_dir)
   elif classifier_name == 'pretrain_Bart':
        classifier = BartForSequenceClassification.from_pretrained('facebook/bart-large', num_labels=3,cache_dir=customed_cache_dir)
   elif classifier_name == 'pretrain_RoBerta':
        classifier = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3,cache_dir=customed_cache_dir)

  
    
   classifier.load_state_dict(torch.load(classifier_checkpoint))
    
   model = ZeroShotSentimentDiscovery(decoder, classifier, decoder_tokenizer, sentiment_tokenizer, device = device)
   model.to(device)

# if model_name != 'ZeroShotSentimentDiscovery':
#     # load model and send to device
#     model.load_state_dict(torch.load(checkpoint_path))
#     model.to(device)

''' set up dataloader '''
# test dataset
test_set = ZuCo_dataset(whole_dataset_dict, 'test', tokenizer, subject = subject_choice, eeg_type = eeg_type_choice, bands = bands_choice, setting = 'unique_sent')

dataset_sizes = {'test': len(test_set)}
# print('[INFO]train_set size: ', len(train_set))
print('[INFO]test_set size: ', len(test_set))

test_dataloader = DataLoader(test_set, batch_size = 1, shuffle=False, num_workers=4)
# dataloaders
dataloaders = {'test':test_dataloader}

''' set up optimizer and scheduler'''
optimizer_step1 = None
exp_lr_scheduler_step1 = None

''' set up loss function '''
criterion = nn.CrossEntropyLoss()

print('=== start training ... ===')
# return best loss model from step1 training
model = eval_model(dataloaders, device, model, criterion, optimizer_step1, exp_lr_scheduler_step1, num_epochs=num_epochs, tokenizer = tokenizer)


[INFO] eval ZeroShotSentimentDiscovery
[INFO] using decoder: BrainTranslator
[INFO] using classifier: pretrain_RoBerta
![Debug]using ALL
[INFO]eeg type GD
[INFO]using bands ['_t1', '_t2', '_a1', '_a2', '_b1', '_b2', '_g1', '_g2']
[INFO]using device cuda:1
[INFO]Model: ZeroShotSentimentDiscovery, using classifer:pretrain_RoBerta, using generator: BrainTranslator


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[INFO]loading 1 task datasets
[INFO]using subjects:  ['ZAB', 'ZDM', 'ZDN', 'ZGW', 'ZJM', 'ZJN', 'ZJS', 'ZKB', 'ZKH', 'ZKW', 'ZMG', 'ZPH']
train divider = 320
dev divider = 360
[INFO]initializing a test set...
++ adding task to dataset, now we have: 456
[INFO]input tensor size: torch.Size([56, 840])

[INFO]test_set size:  456
=== start training ... ===
Epoch 0/0
----------

target string: <s>Everything its title implies, a standard-issue crime drama spat out from the Tinseltown assembly line.
predict string:  the a.- the. the. the the's's's. the's the the the's. the.'s's's.. the's the's's the's the's's.'s. the the's the the

target string: <s>This odd, poetic road movie, spiked by jolts of pop music, pretty much takes place in Morton's ever-watchful gaze -- and it's a tribute to the actress, and to her inventive director, that the journey is such a mesmerizing one.
predict string: .. the. the..

target string: <s>Co-writer/director Jonathan Parker's attempts to fashion a Brazil-like, hy

/home/linal20/.conda/envs/pytorch2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
sklearn macro: precision, recall, F1:
(0.2997782705099778, 0.33522391004142826, 0.1661331199146325, None)

sklearn micro: precision, recall, F1:
(0.3026315789473684, 0.3026315789473684, 0.3026315789473684, None)

sklearn accuracy:
0.3026315789473684

In [3]:
decoder.load_state_dict(torch.load(decoder_checkpoint))

NameError: name 'decoder' is not defined

In [8]:
precision = 0.5913511999644029
recall = 0.5849962641007417
F1 = 2 * (precision * recall) / (precision + recall)
F1

0.588156566521928

In [18]:
import os
os.getcwd()

'/home/linal20/NLP_final_project'